## makemore: becoming a backprop ninja

In [1]:
# there no change change in the first several cells from last lecture

In [2]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures
%matplotlib inline

In [3]:
# download the names.txt file from github
!wget https://raw.githubusercontent.com/karpathy/makemore/master/names.txt

--2025-03-02 11:07:05--  https://raw.githubusercontent.com/karpathy/makemore/master/names.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 228145 (223K) [text/plain]
Saving to: ‘names.txt.21’

names.txt.21        100%[===================>] 222.80K  --.-KB/s    in 0.08s   

2025-03-02 11:07:05 (2.86 MB/s) - ‘names.txt.21’ saved [228145/228145]



In [4]:
# read in all the words
words = open('names.txt', 'r').read().splitlines()
print(len(words))
print(max(len(w) for w in words))
print(words[:8])

32033
15
['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']


In [5]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
vocab_size = len(itos)
print(itos)
print(vocab_size)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}
27


In [6]:
# build the dataset
block_size = 3 # context length: how many characters do we take to predict the next one?

def build_dataset(words):
    X, Y = [], []

    for w in words:
        context = [0] * block_size
        for ch in w + '.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            context = context[1:] + [ix] # crop and append

    X = torch.tensor(X)
    Y = torch.tensor(Y)
    print(X.shape, Y.shape)
    return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr,  Ytr  = build_dataset(words[:n1])     # 80%
Xdev, Ydev = build_dataset(words[n1:n2])   # 10%
Xte,  Yte  = build_dataset(words[n2:])     # 10%

torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


In [7]:
# ok biolerplate done, now we get to the action:

In [8]:
# utility function we will use later when comparing manual gradients to PyTorch gradients
def cmp(s, dt, t):
    ex = torch.all(dt == t.grad).item()
    app = torch.allclose(dt, t.grad)
    maxdiff = (dt - t.grad).abs().max().item()
    print(f'{s:15s} | exact: {str(ex):5s} | approximate: {str(app):5s} | maxdiff: {maxdiff}')

In [9]:
n_embd = 10 # the dimensionality of the character embedding vectors
n_hidden = 64 # the number of neurons in the hidden layer of the MLP

g = torch.Generator().manual_seed(2147483647) # for reproducibility
C  = torch.randn((vocab_size, n_embd),            generator=g)
# Layer 1
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) * (5/3)/((n_embd * block_size)**0.5)
b1 = torch.randn(n_hidden,                        generator=g) * 0.1 # using b1 just for fun, it's useless because of BN
# Layer 2
W2 = torch.randn((n_hidden, vocab_size),          generator=g) * 0.1
b2 = torch.randn(vocab_size,                      generator=g) * 0.1
# BatchNorm parameters
bngain = torch.randn((1, n_hidden))*0.1 + 1.0
bnbias = torch.randn((1, n_hidden))*0.1

# Note: I am initializating many of these parameters in non-standard ways
# because sometimes initializating with e.g. all zeros could mask an incorrect
# implementation of the backward pass.

parameters = [C, W1, b1, W2, b2, bngain, bnbias]
print(sum(p.nelement() for p in parameters)) # number of parameters in total
for p in parameters:
    p.requires_grad = True

4137


In [10]:
batch_size = 32
n = batch_size # a shorter variable also, for convenience
# construct a minibatch
ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
Xb, Yb = Xtr[ix], Ytr[ix] # batch X,Y

#### (1) dlogprobs

We need to obtain the derivative dloss / dlogprobs. This requires no chain rule since it's the very first gradient in the backpropogration.

We need to unravel what the loss is doing: loss = -logprobs[range(n), Yb].mean()

logprobs[range(n), Yb] is forming a new tensor where each successive element corresponds to the element of Yb, i.e. logprobs[0][Yb[0]] + logprobs[1][Yb[1]] + ... + logprobs[31][Yb[31]] since n = 32. We divide this by 32. When taking the derivative of each element w.r.t logprobs, we're left with the normalization factor 1/32. Putting the negative back in we get -1/32. This also means that every element in logprobs that isn't directly associated with the loss has no gradient since it won't contribute to the output of loss

What this means is that all elements in dlogprobs will be zero except for the ones that do directly contribute. Andrej gave the very simple example of this in his solution.

loss = - (a + b + c) / 3 = -a/3 -b/3 -c/3, so for each element in logprobs this looks like -1/3 . In our case, 3->32 so every element should be -1/32 that isn't zero. 

we write this in one line as: `dlogprobs = torch.zeros_like(logprobs); dlogprobs[range(n), Yb] = -1.0/n`

#### (2) dprobs

Now we have to start using the chain rule to backpropogate through the network

dloss / dprobs = dloss / dlogprobs * dlogprobs / dprobs

the first term on the rhs was just calculated, so all we need to do is calculate the second term. I'm going to use the derivative of the log for this one:

dlogprobs / dprobs = d(log(probs)) / dprobs = 1 / probs

This means I'm going to invert all of my elements in probs, and this completes this term.

Now we have to figure out how to combine these together. We can either combine them using @ or *, which are matrix multiplication and Hadamard product (element wise multiplication) respectively

Using @, this won't work since (dloss / dlogoprobs).shape == (dlogprobs / dprobs).shape = (32, 27). We can use tensor broadcasting for this though, where we just have to make sure that the dimensions are aligned properly

(align from the right)

(32, 27)
(32, 27)

and this will work! From: https://pytorch.org/docs/stable/notes/broadcasting.html

In [11]:
## Examples for broadcasting: 
# RULES: (1) each tensor must have dimension 1
#        (2) starting at trailing dimension, each dimension must be equal, be 1, or DNE
# final tensor will be the shape of the tensor with larger dimension

In [12]:
a = torch.tensor([1, 2, 3])
b = torch.tensor([[5, 5, 5], [6, 6, 6]])
c = torch.tensor([[10], [20], [30]])
d = torch.tensor([[5, 5], [6, 6], [7, 7], [8, 8]])
e = torch.tensor([[9], [9], [9], [9], [9]])

# a * b 
# (3) ====> (1, 3) Broadcastable ==> (2, 3) Final shape = (2, 3)
# (2, 3) => (2, 3) ================> (2, 3)
# a * b = [[1*5, 2*5, 3*5], [1*6, 2*6, 3*6]]

# a * c
# (3) ====> (1, 3) Broadcastable ==> (3, 3) Final shape = (3, 3) 
# (3, 1) => (3, 1) ================> (3, 1)
# a * c = [[1*10, 2*10, 3*10], [1*20, 2*20, 3*20], [1*30, 2*30, 3*30]]

# c * a
# (3, 1) ==> (3, 1) Broadcastable ==> (3, 1)
# (3) =====> (1, 3) ================> (3, 3)

# d * a 
# (4, 2) => (4,   2) NOT broadcastable since 2 =/= 3
# (3) ====> (DNE, 3)

# a * d
# (3) =====> (DNE, 3) NOT broadcastable since 3 =/= 2
# (4, 2) ==> (4,   2)

# b * e
# (2, 3) NOT broadcastable since 2 =/= 5
# (5, 1)

# c * e
# (3, 1) ==> NOT broadcastable since 3 =/= 5
# (5, 1)
# -----

#### (3) dcounts_sum_inv

dloss / dcsi = dloss / dprobs * dprobs / dcsi

1st term on RHS we already got, but dprobs/dcsi needs to be expanded: d(counts * csi)/dcsi = counts

this is the correct answer for the element-wise derivative. but we used csi multiple times (3) in order to broadcast the tensor properly. we see this after taking a look at counts * dprobs vs. shape of csi

Remember that the derivative of whatever you were differentiating must have the same shape in order to properly account for the gradient. This means that dprobs * counts is definitely not the complete answer in this case. And it turns out it's because we have yet to account for the broadcasting that occured in order to obtain probs, which is counts * counts_sum_inv

Remember from micrograd that when we were broadcasting vectors within our network, this amounted to adding up however many times this happened

each csi element is the same variable that is used 32 times in order to propogate the gradient through the network, and these gradients summed together in the micrograd case. This would take the 27 appearences of the same variable within each "use" and sum it all together. We haven't done this summation trick yet, and we implement it by using (dprob * counts).sum(dim=1, keepdims=True)

To back propogate through replication, we sum the gradients together within the rows---which gave the info. about how many times a column was utilized during a forward pass

You can kind of think about the difference between "how much did this one variable change the outcome each time it was replicated", so you have to not only think about the element-wise multiplication, but also think about what happens to the gradients. It's almost like you're differentiating a vector function v = x(t), y(t), z(t)), which you do by: dv/dt = dx/dt xhat + dy/dt yhat + dz/dt zhat where because we've already normalized this vector function at this point, we already have unit vectors, which is why we add them all together. this is kind of like what's happening  in our case because we sum all of the gradients that are involved in this tensor broadcast, so really we're kind of looking at dprobs/dcsi = ([[dcounts_i/dcsi_i + ...], [...]).sum(dim=1, keepdim=True) so that we maintain the structure of the tensor so that it matches prob. 

There are multiple ways to think about it, but whenever we are using replication via tensor broadcasting, we must account for that within our back propogation!

#### (4) dcounts_sum

dloss / dcounts_sum = dloss / dcounts_sum_inv * dcounts_sum_inv / dcounts_sum

A good rule of thumb: (1) always check the shapes of your tensors, so that you have an idea about what's going on

dloss / dcounts_sum_inv we already have.

The shapes of dcounts_sum_inv and counts_sum are the same, so there's no broadcasting, and therefore no replication of tensors happening in the backprop

This means we can just treat this local derivative like a standard variable

dcounts_sum_inv / dcounts_sum = d/dcounts_sum(counts_sum**-1) = -counts_sum**-2

#### (5) dcounts

counts is a little tricky since it was used twice---it's used in order to get probs, as well as counts_sum_inv, so there will be two contributions to its derivative due to being used twice. There will be a contribution from probs, as well as another contribution from counts_sum

Andrej gives this to us, but it's found through:

dloss/dcounts = dloss/dprobs * dprobs/dcounts + dloss/dcounts_sum * dcounts_sum/dcounts.

First term is given * counts_sum_inv, and second term is below:

for counts_sum, this is adding up all of the elements within each vector of counts. this is equivalent of the following case for the first row: 

counts = (a, b, c) ; counts.shape = (1, 3)
d(a + b + c) / dcounts = d(a)/dcounts + d(b)/dcounts + d(c)/dcounts = (1, 0, 0) + (0, 1, 0) + (0, 0, 1) = (1,1,1)

This is true because, for some function f and vector x = (x1, x2, x3), grad(f) = (df/dx1, df/dx2, df/dx3)

This will be true for all vectors within counts, so this means we'll have an entire (32, 27) matrix of ones. And this checks out

Andrej's explanation goes like: Addition is a router for the gradient to travel within the row vectors. Each row vector will have all 1's corresponding to the proper gradient, while all others will be zero. This will be true for each deriavative

I'm starting to get a general feel for what's going on and how to understand what's going on in the network: First check the shapes of the tensors you're working with. This will help you determine if there's some kind of broadcasting happening. Next is to create simple examples of what's happening, which will help you understand the fundamentals of a particular operation. Third is to scale up to the actual problem with both of these

#### (6) dnorm_logits

This one's easy since there's an exponential for the local derivative

#### (7) dlogit_maxes

This one was trickier. The element-wise local derivative is all negative ones, since this is what we were subtracting from logits

Also, the shapes are all over the place. logit_maxes.shape = [32, 1], dnorm_logits.shape = [32, 27] = logits.shape

Also: `dlogit_maxes = dnorm_logits.sum(dim=1, keepdim=True)*(-torch.ones_like(logit_maxes))`

can be just easily turned into: `-dlogit_maxes = -dnorm_logits.sum(dim=1, keepdim=True)`

#### (8) dlogits

I got this on the first try, and it was a two parter. I really feel like a ninja lmao

The first term on RHS is simpler than the second; the local derivative will just be all ones since that's all what's relevant in norm_logits = logits - logit_maxes

The second RHS term is trickier. We can use an easier example to understand what's going on

let x = [[x11, x12], [x21, x22]] and xmax = [x11, x22] for example. Then when we take

dxmax/dx = d/dx ([x11, x22]) = [[d/d(x11, x12)(x11), d/d(x11, x12)(x22)], [d/d(x21, x22)(x11), d/d(x21, x22)(x22)], this will create 1s wherever the maximum arg appears, but 0 everywhere else. Thinking in terms of coordinate vector spaces really helped me out here 

= [[d/d(x11, x12)(x11), d/d(x11, x12)(x22)], [d/d(x21, x22)(x11), d/d(x21, x22)(x22)]

= [[((1, 0), (0, 0)), ((0, 0), (0, 1))] ==> [[1, 0], [0, 1]] after "summing" the most internal parentheses tuples ==> really it's representing only the non-zero "phase space coordinates" of the vectors this way

This means that we just need to pluck out the indices of whereever the maximum element appears, and then set this to 1. I definitely had an inefficient way of doing this but I don't know how to optimize yet

`imax = torch.tensor([torch.argmax(logits[i]) for i in range(n)])`

`x = torch.zeros_like(logits); x[range(n), imax] = 1`

AND THIS WORKED!

And indeed, Andrej has a better solution

using .max() also returns the indices! it's under logits.max(dim=1, keepdim=True).indices

`# this gets the EXACT result, and was inspired by what we did previously!`

`imax = torch.tensor([torch.argmax(logits[i]) for i in range(n)])`

`x = torch.zeros_like(logits); x[range(n), imax] = 1`

`# this gets the approximate result, but still clunky`

`x = torch.zeros_like(logits); x[range(n), logits.max(dim=1, keepdim=True).indices] = 1`

Also for the first term RHS, an equivalent of `dnorm_logits*torch.ones_like(logits)` is `torch.clone(dnorm_logits)`, which I'm assuming is more energy efficient

The Andrej way is: `F.one_hot(logits.max(dim=1).indices, num_classes=logits.shape[1])`, where the first argument is the indices where the maximum element occurs, and num_classes specifies the number of elements there will be in the row vector, so there will be a 1 wherever the max elem appeared in that index, and a 0 everywhere else in the row vector

#### (9) dh

Turns out I realize that I've never take the derivative of a matrix with respect to a matrix before, so this was an entirely new learning experience. It's much easier to first work out the details with a 2x2 matrix and then take the derivative of each element. For example consider the 2x2 matrix $\mathbf{D} = \mathbf{A}\mathbf{B} + \mathbf{C}$ and then work out the individual terms. The first element in the first row and column looks like

$\mathrm{d}\mathbf{D}/\mathrm{d}\mathbf{A} = \mathrm{d}d_{11}/\mathrm{d}a_{11} + \mathrm{d}d_{12}/\mathrm{d}a_{11} = b_{11} \mathrm{d}d_{11}/\mathrm{d}a_{11} + b_{12} \mathrm{d}d_{12}/\mathrm{d}a_{11}$

since the there are two terms which are dependent on $a_{11}$. Eventually what this looks like is:

$\mathrm{d}\mathbf{D}/\mathrm{d}\mathbf{A}$ = $\mathrm{d}\mathbf{D}/\mathrm{d}\mathbf{A} @ B^T$

and the final formula would look like:

$\mathrm{d}Loss/\mathrm{d}h = \mathrm{d}Loss / \mathrm{d}logits @ \mathbf{W2}^T$

I'm not going to write out all of the other matrix elements

#### (10) dW2

Similar concept, except with Andrej's secret: Transpose the chain rule term, and then matrix multiply by dLoss/dlogits. Easy.

#### (11)

Same thing, need to eliminate the dimension so that the dimensions of the tensors match. So just sum up all of the rows of dlogits so that the dimensions match up!

#### (12) dhpreact

Pretty simple since $d(tanh(x))/dx = 1 - tanh^2(x)$ , and multiply this by the chain rule factor

#### (13) dbngain

Back to being tricky again. It first helps to look at the dimensions of things

`dhpreact.shape = (32, 64), bngain.shape = (1, 64)`

First we know by chain rule that there is a dhpreact and bnraw since every element contains an element wise multiplication of bngain and bnraw, which are linearly proportional. So the initial answer is bnraw * dhpreact. But now the dimensions don't add up--this is because bngain was broadcasted in the dim=0 dimension since you align dimensions on the right when you use tensor broadcasting. This means that we need to sum along this dimension in order to get the correct answer


#### (14) dbnraw

Same kind of thing here, but since the dimension of bnraw is (32, 64) you don't need to sum along any dimension since there was no broadcasting happening

#### (15) dbnbias

Since bnbias is linear via addition in hpreact, we just need to sum along the appropriate dimension, that being dim=0 and keepdim=True as usual

#### (16) dbnvar_inv

Ditto to previous

#### (17) dbnvar 

Standard derivative taking with chain rule, nothing crazy

#### (18) dbndiff2

A little tricky here with the sum along dim=0. dbnvar / dbndiff2 will give a torch.ones(bndiff2.shape) with elements being 32, but we are dividing by n-1, so no need to add it here since we're already having this normalizing part 

#### (19)-(25)  bndiff - emb

I did all of these in the span of 10-15 mins, I do feel like a backprop ninja now.

Be wary of unbiased vs. bias variance calculations--better to stick with BatchNorm

#### (26) C

This was tricky forsure and it's also something I've never done before.

The idea is that we want to pass in the gradients to each row of C, which corresponds to a 10-dimensional embedding of a character, depending on when it shows up in the batch. Xb tells us which row we should be depositing the gradient, and demb is the gradient that we need to pass through

For example , `Xb[0] = tensor[1,1,4]` which tells us that we need to deposit gradients of demb corresponding to the first, first and fourth rows of demb into the first, first and fourth rows of dC. These gradients need to be additive as well, otherwise we won't capture the entire chain rule.

Andrej says there's not a vectorized version of this so I'll take his word on it

In [13]:
# forward pass, "chunkated" into smaller steps that are possible to backward one at a time
emb = C[Xb] # embed the characters into vectors
embcat = emb.view(emb.shape[0], -1) # concatenate the vectors
# Linear layer 1
hprebn = embcat @ W1 + b1 # hidden layer pre-activation
# BatchNorm layer
bnmeani = 1/n*hprebn.sum(0, keepdim=True)
bndiff = hprebn - bnmeani
bndiff2 = bndiff**2
bnvar = 1/(n-1)*(bndiff2).sum(0, keepdim=True) # note: Bessel's correction (dividing by n-1, not n)
bnvar_inv = (bnvar + 1e-5)**-0.5
bnraw = bndiff * bnvar_inv
hpreact = bngain * bnraw + bnbias
# Non-linearity
h = torch.tanh(hpreact) # hidden layer
# Linear layer 2
logits = h @ W2 + b2 # output layer
# cross entropy loss (same as F.cross_entropy(logits, Yb))
logit_maxes = logits.max(1, keepdim=True).values
norm_logits = logits - logit_maxes # subtract max for numerical stability
counts = norm_logits.exp()
counts_sum = counts.sum(dim=1, keepdims=True)
counts_sum_inv = counts_sum**-1 # if I use (1.0 / counts_sum) instead then I can't get backprop to be bit exact...
probs = counts * counts_sum_inv
logprobs = probs.log()
loss = -logprobs[range(n), Yb].mean()

# PyTorch backward pass
for p in parameters:
    p.grad = None
for t in [logprobs, probs, counts, counts_sum, counts_sum_inv, # afaik there is no cleaner way
          norm_logits, logit_maxes, logits, h, hpreact, bnraw,
         bnvar_inv, bnvar, bndiff2, bndiff, hprebn, bnmeani,
         embcat, emb]:
    t.retain_grad()
loss.backward()
loss

tensor(3.3519, grad_fn=<NegBackward0>)

In [14]:
# Exercise 1: backprop through the whole thing manually,
# backpropagating through exactly all of the variables
# as they are defined in the forward pass above, one by one

# cross entropy loss
dlogprobs = torch.zeros_like(logprobs); dlogprobs[range(n), Yb] = -1.0/n
dprobs = dlogprobs * (1 / probs)
dcounts_sum_inv = (dprobs * counts).sum(dim=1, keepdim=True)
dcounts_sum = dcounts_sum_inv * (-counts_sum**(-2))
dcounts = dprobs*counts_sum_inv + dcounts_sum*torch.ones_like(counts)
dnorm_logits = dcounts*counts.clone()
dlogit_maxes = -dnorm_logits.sum(dim=1, keepdim=True)
dlogits = torch.clone(dnorm_logits) 
dlogits += F.one_hot(logits.max(dim=1).indices, num_classes=logits.shape[1])*dlogit_maxes

# second linear layer
dh = dlogits @ W2.T
dW2 = h.T @ dlogits
db2 = dlogits.sum(dim=0, keepdim=True)

# non-linearity
dhpreact = (1.0 - h.clone()**2) * dh

# batch-norm layer
dbngain = (bnraw * dhpreact).sum(dim=0, keepdim=True)
dbnraw = bngain * dhpreact
dbnbias = dhpreact.sum(dim=0, keepdim=True)
dbnvar_inv = (dbnraw * bndiff).sum(dim=0, keepdim=True)
dbnvar = dbnvar_inv * (-0.5 * (bnvar + 1e-5)**(-1.5))
dbndiff2 = dbnvar * (1/(n-1)) * torch.ones(bndiff2.shape)
dbndiff = (dbnraw * bnvar_inv) + (dbndiff2 * 2*bndiff)
dbnmeani = -1.0 * dbndiff.sum(dim=0, keepdim=True)

# first linear layer
dhprebn = dbndiff + dbnmeani * (1/n)

# forward pass
dembcat = dhprebn @ W1.T
dW1 = embcat.T @ dhprebn
db1 = dhprebn.sum(dim=0, keepdim=True)
#demb = dembcat.reshape(emb.shape) # either one works
demb = dembcat.view(emb.shape)
dC = torch.zeros_like(C)
for i, batch in enumerate(Xb):
    for j, elem in enumerate(batch):
        dC[elem.item()] += demb[i][j]

# -----------------
# YOUR CODE HERE :)
# -----------------

cmp('logprobs', dlogprobs, logprobs)
cmp('probs', dprobs, probs)
cmp('counts_sum_inv', dcounts_sum_inv, counts_sum_inv)
cmp('counts_sum', dcounts_sum, counts_sum)
cmp('counts', dcounts, counts)
cmp('norm_logits', dnorm_logits, norm_logits)
cmp('logit_maxes', dlogit_maxes, logit_maxes)
cmp('logits', dlogits, logits)
cmp('h', dh, h)
cmp('W2', dW2, W2)
cmp('b2', db2, b2)
cmp('hpreact', dhpreact, hpreact)
cmp('bngain', dbngain, bngain)
cmp('bnraw', dbnraw, bnraw)
cmp('bnbias', dbnbias, bnbias)
cmp('bnvar_inv', dbnvar_inv, bnvar_inv)
cmp('bnvar', dbnvar, bnvar)
cmp('bndiff2', dbndiff2, bndiff2)
cmp('bndiff', dbndiff, bndiff)
cmp('bnmeani', dbnmeani, bnmeani)
cmp('hprebn', dhprebn, hprebn)
cmp('embcat', dembcat, embcat)
cmp('W1', dW1, W1)
cmp('b1', db1, b1)
cmp('emb', demb, emb)
cmp('C', dC, C)

logprobs        | exact: True  | approximate: True  | maxdiff: 0.0
probs           | exact: True  | approximate: True  | maxdiff: 0.0
counts_sum_inv  | exact: True  | approximate: True  | maxdiff: 0.0
counts_sum      | exact: True  | approximate: True  | maxdiff: 0.0
counts          | exact: True  | approximate: True  | maxdiff: 0.0
norm_logits     | exact: True  | approximate: True  | maxdiff: 0.0
logit_maxes     | exact: True  | approximate: True  | maxdiff: 0.0
logits          | exact: True  | approximate: True  | maxdiff: 0.0
h               | exact: True  | approximate: True  | maxdiff: 0.0
W2              | exact: True  | approximate: True  | maxdiff: 0.0
b2              | exact: True  | approximate: True  | maxdiff: 0.0
hpreact         | exact: False | approximate: True  | maxdiff: 4.656612873077393e-10
bngain          | exact: False | approximate: True  | maxdiff: 2.3283064365386963e-09
bnraw           | exact: False | approximate: True  | maxdiff: 9.313225746154785e-10
bnbias 

In [15]:
counts[0], counts_sum[0]

(tensor([0.7973, 1.0000, 0.1991, 0.4694, 0.2212, 0.9000, 0.2688, 0.3839, 0.2052,
         0.3469, 0.3992, 0.4078, 0.3784, 0.2909, 0.4004, 0.1451, 0.1000, 0.2077,
         0.1929, 0.5982, 0.5619, 0.2456, 0.2918, 0.7349, 0.6318, 0.2797, 0.2334],
        grad_fn=<SelectBackward0>),
 tensor([10.8917], grad_fn=<SelectBackward0>))

In [ ]:
counts.shape, counts_sum.shape

In [30]:
# Exercise 2: backprop through cross_entropy but all in one go
# to complete this challenge look at the mathematical expression of the loss,
# take the derivative, simplify the expression, and just write it out

# forward pass

# before:
# logit_maxes = logits.max(1, keepdim=True).values
# norm_logits = logits - logit_maxes # subtract max for numerical stability
# counts = norm_logits.exp()
# counts_sum = counts.sum(1, keepdims=True)
# counts_sum_inv = counts_sum**-1 # if I use (1.0 / counts_sum) instead then I can't get backprop to be bit exact...
# probs = counts * counts_sum_inv
# logprobs = probs.log()
# loss = -logprobs[range(n), Yb].mean()

# now:
loss_fast = F.cross_entropy(logits, Yb)
print(loss_fast.item(), 'diff:', (loss_fast - loss).item())

3.3132190704345703 diff: 0.0


In [20]:
counts.shape

torch.Size([32, 27])

In [123]:
# backward pass
## Completely misunderstood the asignment: :) --> (: --> :)
''' ## Restarting below this but keeping this for now, getting the wrong answer anyway 
# -----------------
# YOUR CODE HERE :) my solution is 3 lines
t1 = -F.one_hot(logits.argmax(dim=1), num_classes=27)*torch.ones_like(logit_maxes) + torch.ones_like(logits)

t2 = counts*(-1.0*torch.ones_like(counts)*counts_sum**-2 + counts_sum_inv)

dlogits = -(1.0/n)*F.one_hot(Yb, num_classes=27)*(1/probs)*norm_logits.exp() * t1 * t2
# -----------------

cmp('logits', dlogits, logits) # I can only get approximate to be true, my maxdiff is 6e-9
'''

logits          | exact: False | approximate: False | maxdiff: 0.028187651187181473


In [33]:
ex = torch.zeros_like(logits)

ex[range(n), Yb] = (-1/n) * counts_sum_inv * (counts_sum - counts[range(n), Yb])
ex

RuntimeError: shape mismatch: value tensor of shape [32, 32] cannot be broadcast to indexing result of shape [32]

In [26]:
# backward pass

# -----------------
# YOUR CODE HERE :) my solution is 3 lines
dlogits = torch.zeros_like(counts)
dlogits[range(n), Yb] = -(n-1/n) 

# -----------------

cmp('logits', dlogits, logits) # I can only get approximate to be true, my maxdiff is 6e-9

logits          | exact: False | approximate: False | maxdiff: 31.942655563354492


In [124]:
probs

tensor([[0.0714, 0.0870, 0.0177, 0.0502, 0.0194, 0.0835, 0.0224, 0.0350, 0.0188,
         0.0324, 0.0339, 0.0367, 0.0385, 0.0292, 0.0363, 0.0138, 0.0088, 0.0198,
         0.0164, 0.0585, 0.0489, 0.0209, 0.0250, 0.0686, 0.0612, 0.0249, 0.0208],
        [0.0531, 0.0587, 0.0961, 0.0588, 0.0358, 0.0319, 0.0183, 0.0508, 0.0213,
         0.0231, 0.0504, 0.0379, 0.0470, 0.0309, 0.0448, 0.0381, 0.0238, 0.0168,
         0.0200, 0.0442, 0.0181, 0.0216, 0.0157, 0.0546, 0.0242, 0.0402, 0.0238],
        [0.0202, 0.0250, 0.0147, 0.0148, 0.0237, 0.0404, 0.0533, 0.0629, 0.0623,
         0.0316, 0.0195, 0.0326, 0.0453, 0.0517, 0.0246, 0.0267, 0.0141, 0.0347,
         0.0280, 0.1128, 0.0652, 0.0329, 0.0387, 0.0378, 0.0394, 0.0194, 0.0277],
        [0.0324, 0.0271, 0.0405, 0.0588, 0.0555, 0.0279, 0.0443, 0.0435, 0.0520,
         0.0175, 0.0348, 0.0248, 0.0331, 0.0438, 0.0554, 0.0639, 0.0243, 0.0258,
         0.0158, 0.0587, 0.0176, 0.0241, 0.0385, 0.0190, 0.0268, 0.0504, 0.0436],
        [0.0169, 0.0162,

In [150]:
# Exercise 3: backprop through batchnorm but all in one go
# to complete this challenge look at the mathematical expression of the output of batchnorm,
# take the derivative w.r.t. its input, simplify the expression, and just write it out
# BatchNorm paper: https://arxiv.org/abs/1502.03167

# forward pass

# before:
# bnmeani = 1/n*hprebn.sum(0, keepdim=True)
# bndiff = hprebn - bnmeani
# bndiff2 = bndiff**2
# bnvar = 1/(n-1)*(bndiff2).sum(0, keepdim=True) # note: Bessel's correction (dividing by n-1, not n)
# bnvar_inv = (bnvar + 1e-5)**-0.5
# bnraw = bndiff * bnvar_inv
# hpreact = bngain * bnraw + bnbias

# now:
hpreact_fast = bngain * (hprebn - hprebn.mean(0, keepdim=True)) / torch.sqrt(hprebn.var(0, keepdim=True, unbiased=True) + 1e-5) + bnbias
print('max diff:', (hpreact_fast - hpreact).abs().max())

max diff: tensor(4.7684e-07, grad_fn=<MaxBackward1>)


In [ ]:
# backward pass

# before we had:
# dbnraw = bngain * dhpreact
# dbndiff = bnvar_inv * dbnraw
# dbnvar_inv = (bndiff * dbnraw).sum(0, keepdim=True)
# dbnvar = (-0.5*(bnvar + 1e-5)**-1.5) * dbnvar_inv
# dbndiff2 = (1.0/(n-1))*torch.ones_like(bndiff2) * dbnvar
# dbndiff += (2*bndiff) * dbndiff2
# dhprebn = dbndiff.clone()
# dbnmeani = (-dbndiff).sum(0)
# dhprebn += 1.0/n * (torch.ones_like(hprebn) * dbnmeani)

# calculate dhprebn given dhpreact (i.e. backprop through the batchnorm)
# (you'll also need to use some of the variables from the forward pass up above)

# -----------------
# YOUR CODE HERE :)
dhprebn = None # TODO. my solution is 1 (long) line
# -----------------

cmp('hprebn', dhprebn, hprebn) # I can only get approximate to be true, my maxdiff is 9e-10

In [ ]:
# Exercise 4: putting it all together!
# Train the MLP neural net with your own backward pass

# init
n_embd = 10 # the dimensionality of the character embedding vectors
n_hidden = 200 # the number of neurons in the hidden layer of the MLP

g = torch.Generator().manual_seed(2147483647) # for reproducibility
C  = torch.randn((vocab_size, n_embd),            generator=g)
# Layer 1
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) * (5/3)/((n_embd * block_size)**0.5)
b1 = torch.randn(n_hidden,                        generator=g) * 0.1
# Layer 2
W2 = torch.randn((n_hidden, vocab_size),          generator=g) * 0.1
b2 = torch.randn(vocab_size,                      generator=g) * 0.1
# BatchNorm parameters
bngain = torch.randn((1, n_hidden))*0.1 + 1.0
bnbias = torch.randn((1, n_hidden))*0.1

parameters = [C, W1, b1, W2, b2, bngain, bnbias]
print(sum(p.nelement() for p in parameters)) # number of parameters in total
for p in parameters:
  p.requires_grad = True

# same optimization as last time
max_steps = 200000
batch_size = 32
n = batch_size # convenience
lossi = []

# use this context manager for efficiency once your backward pass is written (TODO)
#with torch.no_grad():

# kick off optimization
for i in range(max_steps):

  # minibatch construct
  ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
  Xb, Yb = Xtr[ix], Ytr[ix] # batch X,Y

  # forward pass
  emb = C[Xb] # embed the characters into vectors
  embcat = emb.view(emb.shape[0], -1) # concatenate the vectors
  # Linear layer
  hprebn = embcat @ W1 + b1 # hidden layer pre-activation
  # BatchNorm layer
  # -------------------------------------------------------------
  bnmean = hprebn.mean(0, keepdim=True)
  bnvar = hprebn.var(0, keepdim=True, unbiased=True)
  bnvar_inv = (bnvar + 1e-5)**-0.5
  bnraw = (hprebn - bnmean) * bnvar_inv
  hpreact = bngain * bnraw + bnbias
  # -------------------------------------------------------------
  # Non-linearity
  h = torch.tanh(hpreact) # hidden layer
  logits = h @ W2 + b2 # output layer
  loss = F.cross_entropy(logits, Yb) # loss function

  # backward pass
  for p in parameters:
    p.grad = None
  loss.backward() # use this for correctness comparisons, delete it later!

  # manual backprop! #swole_doge_meme
  # -----------------
  # YOUR CODE HERE :)
  dC, dW1, db1, dW2, db2, dbngain, dbnbias = None, None, None, None, None, None, None
  grads = [dC, dW1, db1, dW2, db2, dbngain, dbnbias]
  # -----------------

  # update
  lr = 0.1 if i < 100000 else 0.01 # step learning rate decay
  for p, grad in zip(parameters, grads):
    p.data += -lr * p.grad # old way of cheems doge (using PyTorch grad from .backward())
    #p.data += -lr * grad # new way of swole doge TODO: enable

  # track stats
  if i % 10000 == 0: # print every once in a while
    print(f'{i:7d}/{max_steps:7d}: {loss.item():.4f}')
  lossi.append(loss.log10().item())

  if i >= 100: # TODO: delete early breaking when you're ready to train the full net
    break

In [ ]:
# useful for checking your gradients
# for p,g in zip(parameters, grads):
#   cmp(str(tuple(p.shape)), g, p)

In [ ]:
# calibrate the batch norm at the end of training

with torch.no_grad():
  # pass the training set through
  emb = C[Xtr]
  embcat = emb.view(emb.shape[0], -1)
  hpreact = embcat @ W1 + b1
  # measure the mean/std over the entire training set
  bnmean = hpreact.mean(0, keepdim=True)
  bnvar = hpreact.var(0, keepdim=True, unbiased=True)


In [ ]:
# evaluate train and val loss

@torch.no_grad() # this decorator disables gradient tracking
def split_loss(split):
  x,y = {
    'train': (Xtr, Ytr),
    'val': (Xdev, Ydev),
    'test': (Xte, Yte),
  }[split]
  emb = C[x] # (N, block_size, n_embd)
  embcat = emb.view(emb.shape[0], -1) # concat into (N, block_size * n_embd)
  hpreact = embcat @ W1 + b1
  hpreact = bngain * (hpreact - bnmean) * (bnvar + 1e-5)**-0.5 + bnbias
  h = torch.tanh(hpreact) # (N, n_hidden)
  logits = h @ W2 + b2 # (N, vocab_size)
  loss = F.cross_entropy(logits, y)
  print(split, loss.item())

split_loss('train')
split_loss('val')

In [ ]:
# I achieved:
# train 2.0718822479248047
# val 2.1162495613098145

In [ ]:
# sample from the model
g = torch.Generator().manual_seed(2147483647 + 10)

for _ in range(20):

    out = []
    context = [0] * block_size # initialize with all ...
    while True:
      # forward pass
      emb = C[torch.tensor([context])] # (1,block_size,d)
      embcat = emb.view(emb.shape[0], -1) # concat into (N, block_size * n_embd)
      hpreact = embcat @ W1 + b1
      hpreact = bngain * (hpreact - bnmean) * (bnvar + 1e-5)**-0.5 + bnbias
      h = torch.tanh(hpreact) # (N, n_hidden)
      logits = h @ W2 + b2 # (N, vocab_size)
      # sample
      probs = F.softmax(logits, dim=1)
      ix = torch.multinomial(probs, num_samples=1, generator=g).item()
      context = context[1:] + [ix]
      out.append(ix)
      if ix == 0:
        break

    print(''.join(itos[i] for i in out))